In [1]:
from src.densite_function import *
from src.gaussian_simulation import *
from src.estimators import *
from src.vraisemblance import *
import numpy as np
from numpy.linalg import norm 
import matplotlib.pylab as plt

In [2]:
np.random.seed(8878)

theta=simulate_gaussian_vector(mu=np.array([0]*20), sigma=np.identity(20))

A_optimal=np.matrix(np.identity(20))*0.5
b_optimal=theta/2

A=A_optimal+simulate_gaussian_vector(mu=np.array([0]*20), sigma=0.01*np.identity(20))
b=b_optimal+simulate_gaussian_vector(mu=np.array([0]*20), sigma=0.01*np.identity(20))

On généré un n-échantillon de taille 100, de loi $p_{\theta}(\mathbf{x})=\mathcal{N}(\theta, 2I_{20})$

In [3]:
np.random.seed(8878)

def generer_nech_gaussien(n):
    i=1
    echantillon_x=np.array([])

    while i<=n:
        if i==1:
            echantillon_x=np.append(echantillon_x, simulate_gaussian_vector(mu=theta, sigma=2*np.identity(20)))
        else:
            echantillon_x=np.vstack((echantillon_x, simulate_gaussian_vector(mu=theta, sigma=2*np.identity(20))))
        i+=1
    return echantillon_x

echantillon_x=generer_nech_gaussien(100)

La log-vraisemblance de l'échantillon (avec le facteur $-\frac{1}{n}$) est $l_n(\theta)=\frac{1}{n}\sum_{i=1}^n \lVert X_i - \theta \rVert^2 = \frac{1}{n}\sum_{i=1}^n l_i(\theta) $. 
On cherche $\hat{\theta} \in argmin \; l_n(\theta)$

# SDG

On estime le theta de la log vraisemblance par $\nabla_{\theta} l_i(\theta)= -2(X_i-\theta)$

In [4]:
def SDG(theta_init, learn_rate, echantillon, n_iter):
    #Step 1: mélanger l'échantillon
    i=0
    compteur=0
    np.random.shuffle(echantillon)
    theta=theta_init-learn_rate*(-1)*gradient_log_vraisemblance(echantillon[i], theta_init)

    i+=1
    compteur+=1
    #Tant qu'on n'atteint pas le nombre d'itérations fixé, on actualise theta
    while compteur<n_iter:
        # Si on a parcouru tout l'échantillon alors que le nombre d'itérations n'est pas atteint,
        # On remélange l'échantillon, on reparcourt l'échantillon en commençant par le début
        # Le compteur n'est pas réinitialisé
        if i==len(echantillon):
           i=0
           np.random.shuffle(echantillon)
           theta=theta-learn_rate*(-1)*gradient_log_vraisemblance(echantillon[i], theta)
           i+=1
           compteur+=1
        else:
            theta=theta-learn_rate*(-1)*gradient_log_vraisemblance(echantillon[i], theta)
            i+=1
            compteur+=1

    else:
        return theta

In [5]:
def SDG_IAWE(theta_init, learn_rate, n_iter, A, b, echantillon, k=6):
    #Step 1: mélanger l'échantillon
    i=0
    compteur=0
    np.random.shuffle(echantillon)
    theta=theta_init-learn_rate*importance_sampling_gradientlogvraisemblance(k=k, theta=theta_init, A=A, b=b, x=echantillon[i])

    i+=1
    compteur+=1
    #Tant qu'on n'atteint pas le nombre d'itérations fixé, on actualise theta
    while compteur<n_iter:
        # Si on a parcouru tout l'échantillon alors que le nombre d'itérations n'est pas atteint,
        # On remélange l'échantillon, on reparcourt l'échantillon en commençant par le début
        # Le compteur n'est pas réinitialisé
        if i==len(echantillon):
           i=0
           np.random.shuffle(echantillon)
           theta=theta-learn_rate*importance_sampling_gradientlogvraisemblance(k=k, theta=theta, A=A, b=b, x=echantillon[i])
           i+=1
           compteur+=1
        else:
            theta=theta-learn_rate*importance_sampling_gradientlogvraisemblance(k=k, theta=theta, A=A, b=b, x=echantillon[i])
            i+=1
            compteur+=1

    else:
        return theta

In [6]:
def SDG_SUMO(theta_init, learn_rate, n_iter, A, b, echantillon, l=6):
    #Step 1: mélanger l'échantillon
    i=0
    compteur=0
    np.random.shuffle(echantillon)
    theta=theta_init-learn_rate*estimateur_SUMO_gradientlogvraisemblance(theta=theta_init, A=A, b=b, x=echantillon[i], r=0.6, l=l)

    i+=1
    compteur+=1
    #Tant qu'on n'atteint pas le nombre d'itérations fixé, on actualise theta
    while compteur<n_iter:
        # Si on a parcouru tout l'échantillon alors que le nombre d'itérations n'est pas atteint,
        # On remélange l'échantillon, on reparcourt l'échantillon en commençant par le début
        # Le compteur n'est pas réinitialisé
        if i==len(echantillon):
           i=0
           np.random.shuffle(echantillon)
           theta=theta-learn_rate*estimateur_SUMO_gradientlogvraisemblance(theta=theta, A=A, b=b, x=echantillon[i], r=0.6, l=l)
           i+=1
           compteur+=1
        else:
            theta=theta-learn_rate*estimateur_SUMO_gradientlogvraisemblance(theta=theta, A=A, b=b, x=echantillon[i], r=0.6, l=l)
            i+=1
            compteur+=1

    else:
        return theta

In [7]:
def SDG_RR(theta_init, learn_rate, n_iter, A, b, echantillon, l=6):
    #Step 1: mélanger l'échantillon
    i=0
    compteur=0
    np.random.shuffle(echantillon)
    theta=theta_init-learn_rate*estimateur_ML_RR_gradientlogvraisemblance(x=echantillon[i], theta=theta_init, A=A, b=b, r=0.6, l=l)

    i+=1
    compteur+=1
    #Tant qu'on n'atteint pas le nombre d'itérations fixé, on actualise theta
    while compteur<n_iter:
        # Si on a parcouru tout l'échantillon alors que le nombre d'itérations n'est pas atteint,
        # On remélange l'échantillon, on reparcourt l'échantillon en commençant par le début
        # Le compteur n'est pas réinitialisé
        if i==len(echantillon):
           i=0
           np.random.shuffle(echantillon)
           theta=theta-learn_rate*estimateur_ML_RR_gradientlogvraisemblance(x=echantillon[i], theta=theta, A=A, b=b, r=0.6, l=l)
           i+=1
           compteur+=1
        else:
            theta=theta-learn_rate*estimateur_ML_RR_gradientlogvraisemblance(x=echantillon[i], theta=theta, A=A, b=b, r=0.6, l=l)
            i+=1
            compteur+=1

    else:
        return theta

In [8]:
def SDG_SS(theta_init, learn_rate, n_iter, A, b, echantillon, l=6):
    #Step 1: mélanger l'échantillon
    i=0
    compteur=0
    np.random.shuffle(echantillon)
    theta=theta_init-learn_rate*estimateur_ML_SS_gradientlogvraisemblance(x=echantillon[i], theta=theta_init, A=A, b=b, r=0.6, l=l)

    i+=1
    compteur+=1
    #Tant qu'on n'atteint pas le nombre d'itérations fixé, on actualise theta
    while compteur<n_iter:
        # Si on a parcouru tout l'échantillon alors que le nombre d'itérations n'est pas atteint,
        # On remélange l'échantillon, on reparcourt l'échantillon en commençant par le début
        # Le compteur n'est pas réinitialisé
        if i==len(echantillon):
           i=0
           np.random.shuffle(echantillon)
           theta=theta-learn_rate*estimateur_ML_SS_gradientlogvraisemblance(x=echantillon[i], theta=theta, A=A, b=b, r=0.6, l=l)
           i+=1
           compteur+=1
        else:
            theta=theta-learn_rate*estimateur_ML_SS_gradientlogvraisemblance(x=echantillon[i], theta=theta, A=A, b=b, r=0.6, l=l)
            i+=1
            compteur+=1

    else:
        return theta

In [9]:
np.random.seed(878)
theta_sdg=SDG(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, echantillon=echantillon_x)
theta_iawe=SDG_IAWE(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, k=6)
theta_SUMO=SDG_SUMO(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=6)
theta_RR=SDG_RR(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=6)
theta_SS=SDG_SS(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=6)

print("Vraie valeur de theta: {}".format(np.around(theta, 2)))
print("Estimation de theta par descente de gradient stochastique useuelle: {}".format(np.around(theta_sdg, 2)))
print("Estimation de theta par SDG IWAE: {}".format(np.around(theta_iawe, 2)))
print("Estimation de theta par SDG SUMO: {}".format(np.around(theta_SUMO, 2)))
print("Estimation de theta par SDG RR: {}".format(np.around(theta_RR, 2)))
print("Estimation de theta par SDG SS: {}".format(np.around(theta_SS, 2)))

Vraie valeur de theta: [-0.3  -1.01 -0.57  0.53 -0.45 -1.65 -1.66  1.26 -0.5  -2.16  0.18 -0.7
 -0.77  0.53  0.12 -0.5   0.69 -0.35 -0.43  1.35]
Estimation de theta par descente de gradient stochastique useuelle: [-0.03 -1.04 -0.47  0.34 -0.31 -1.87 -1.52  1.08 -0.37 -1.68  0.29 -0.6
 -0.58  0.56  0.16 -0.41  0.55 -0.33 -0.33  1.33]
Estimation de theta par SDG IWAE: [ 0.43  1.9   0.95 -0.43  0.58  2.79  2.31 -1.19  0.73  2.83 -0.18  1.18
  1.31 -0.49 -0.02  0.94 -0.81  0.52  0.89 -1.75]
Estimation de theta par SDG SUMO: [ 0.36  1.99  0.27 -0.68  0.49  2.48  2.44 -0.47  0.76  2.46  0.1   1.11
  1.72 -0.22  0.42  0.59 -0.28  0.75  0.91 -2.04]
Estimation de theta par SDG RR: [-7.86  0.44 -0.76 -3.34 -0.93  1.5   1.48 -4.38 -0.88  2.56 -3.24 -0.83
 -0.58 -3.48 -2.52 -1.38 -3.53 -1.3  -1.18 -4.79]
Estimation de theta par SDG SS: [-23.44   3.57   2.96  -3.76  -0.11   7.28   9.04  -4.26   5.74   8.91
  -4.93   3.28   6.11  -3.8   -2.04   2.04  -3.52   0.14   5.32  -5.92]


In [10]:
def procedure_MC_theta(M, L, theta, A, b, n):
    biais_SDG_M={}
    biais_IWAE_M={}
    biais_SUMO_M={}
    biais_SS_M={}
    biais_RR_M={}

    var_SDG_M={}
    var_IWAE_M={}
    var_SUMO_M={}
    var_SS_M={}
    var_RR_M={}

    l=1
    while l<=L:
        print(l)
        m=1
        estimations_SDG_M_l=np.array([])
        estimations_IWAE_M_l=np.array([])
        estimations_SUMO_M_l=np.array([])
        estimations_SS_M_l=np.array([])
        estimations_RR_M_l=np.array([])


        while m<=M:
            echantillon_x=generer_nech_gaussien(n)
            theta_SDG=SDG(theta_init=np.array([0]*20), learn_rate=0.01, echantillon=echantillon_x, n_iter=100)
            theta_IAWE=SDG_IAWE(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, k=l)
            theta_SUMO=SDG_SUMO(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=l)
            theta_RR=SDG_RR(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=l)
            theta_SS=SDG_SS(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=l)

            if m==1:
                estimations_SDG_M_l=np.append(estimations_SDG_M_l, theta_SDG)
                estimations_IWAE_M_l= np.append(estimations_IWAE_M_l, theta_IAWE)
                estimations_SUMO_M_l=np.append(estimations_SUMO_M_l, theta_SUMO)
                estimations_RR_M_l=np.append(estimations_RR_M_l, theta_RR)
                estimations_SS_M_l=np.append(estimations_SS_M_l, theta_SS)
            
            else:
                estimations_SDG_M_l=np.vstack((estimations_SDG_M_l, theta_SDG))
                estimations_IWAE_M_l= np.vstack((estimations_IWAE_M_l, theta_IAWE))
                estimations_SUMO_M_l=np.vstack((estimations_SUMO_M_l, theta_SUMO))
                estimations_RR_M_l=np.vstack((estimations_RR_M_l, theta_RR))
                estimations_SS_M_l=np.vstack((estimations_SS_M_l, theta_SS))
            m+=1

        biais_SDG_M_l=np.mean(estimations_SDG_M_l, axis=0)-theta
        biais_IWAE_M_l=np.mean(estimations_IWAE_M_l, axis=0)-theta
        biais_IWAE_M_l=np.mean(estimations_IWAE_M_l, axis=0)-theta
        biais_SUMO_M_l=np.mean(estimations_SUMO_M_l, axis=0)-theta
        biais_SS_M_l=np.mean(estimations_SS_M_l, axis=0)-theta
        biais_RR_M_l=np.mean(estimations_RR_M_l, axis=0)-theta

        squared_biais_SDG_M_l=norm(biais_SDG_M_l)**2
        squared_biais_IWAE_M_l=norm(biais_IWAE_M_l)**2
        squared_biais_SUMO_M_l=norm(biais_SUMO_M_l)**2
        squared_biais_SS_M_l=norm(biais_SS_M_l)**2
        squared_biais_RR_M_l=norm(biais_RR_M_l)**2

        var_SDG_M_l=np.mean(norm(estimations_SDG_M_l-np.mean(estimations_IWAE_M_l, axis=0))**2)
        var_IWAE_M_l=np.mean(norm(estimations_IWAE_M_l-np.mean(estimations_IWAE_M_l, axis=0))**2)
        var_SUMO_M_l=np.mean(norm(estimations_SUMO_M_l-np.mean(estimations_SUMO_M_l, axis=0))**2)
        var_SS_M_l=np.mean(norm(estimations_SS_M_l-np.mean(estimations_SS_M_l, axis=0))**2)
        var_RR_M_l=np.mean(norm(estimations_RR_M_l-np.mean(estimations_RR_M_l, axis=0))**2)
        
        biais_SDG_M[l]=squared_biais_SDG_M_l
        biais_IWAE_M[l]=squared_biais_IWAE_M_l
        biais_SUMO_M[l]=squared_biais_SUMO_M_l
        biais_SS_M[l]=squared_biais_SS_M_l
        biais_RR_M[l]=squared_biais_RR_M_l
        
        var_SDG_M[l]=var_SDG_M_l
        var_IWAE_M[l]=var_IWAE_M_l
        var_SUMO_M[l]=var_SUMO_M_l
        var_SS_M[l]=var_SS_M_l
        var_RR_M[l]=var_RR_M_l

        l+=1

    return biais_SDG_M, biais_IWAE_M, biais_SUMO_M, biais_SS_M, biais_RR_M, var_SDG_M,var_IWAE_M, var_SUMO_M, var_SS_M, var_RR_M

In [11]:
np.random.seed(8554)

biais_SDG_M_theta, biais_IWAE_M_theta, biais_SUMO_M_theta, biais_SS_M_theta, biais_RR_M_theta, var_SDG_M_theta, var_IWAE_M_theta, var_SUMO_M_theta, var_SS_M_theta, var_RR_M_theta = procedure_MC_theta(M=1000, 
                                                                                                                                                                                                        L=6, 
                                                                                                                                                                                                        theta=theta, 
                                                                                                                                                                                                        A=A, 
                                                                                                                                                                                                        b=b,
                                                                                                                                                                                                        n=100)

1


c:\Users\yanis\OneDrive\Documents\Monte Carlo\Projet-Monte-Carlo\src\estimators.py:182: RuntimeWarning: invalid value encountered in true_divide
  return num/denom-theta
c:\Users\yanis\OneDrive\Documents\Monte Carlo\Projet-Monte-Carlo\src\estimators.py:251: RuntimeWarning: invalid value encountered in true_divide
  IWAE_O=np.sum(np.array([w_i*z_i for (w_i,z_i) in zip(array_w_O,z_O)]), axis=0)/np.sum(array_w_O)-theta
c:\Users\yanis\OneDrive\Documents\Monte Carlo\Projet-Monte-Carlo\src\estimators.py:252: RuntimeWarning: invalid value encountered in true_divide
  IWAE_E=np.sum(np.array([w_i*z_i for (w_i,z_i) in zip(array_w_E,z_E)]), axis=0)/np.sum(array_w_E)-theta
c:\Users\yanis\OneDrive\Documents\Monte Carlo\Projet-Monte-Carlo\src\estimators.py:253: RuntimeWarning: invalid value encountered in true_divide
  IWAE_OUE=np.sum(np.array([w_i*z_i for (w_i,z_i) in zip(array_w,z)]), axis=0)/np.sum(array_w)-theta


KeyboardInterrupt: 